## Protein-Ligand Analysis

### Introduction

This notebook contains scripts useful for the analysis and plotting of protein-ligand interactions.

Author: Dr Afroditi-Maria Zaki

SBCB, University of Oxford

### Import required packages

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import contacts
import numpy as np
import matplotlib.pyplot as plt
import pylab
from matplotlib import ticker
from matplotlib.ticker import MaxNLocator
from collections import namedtuple
from collections import Counter
from numpy.linalg import norm
print("All required packages are imported")

### Contact Analysis

This script will go through each pair of atoms from two groups, it will calculate their separation and if it is within the cutoff distance, it will count it as a contact. It will then save the fraction of contacts formed vs time.

It can be useful to monitor if a significant change of the binding of the ligand occurs.

You need to provide the path of the input files, the .gro and .xtc (or .trr) files, the group selections (e.g. protein and ligand) and the cutoff.

### User defined input parameters

In [6]:
#Replace with the path to your directory, the name of your .gro and your .xtc (or .trr) files
path = "/home/mjkikaz2/Documents/WORK/Mycolactone/MUTATIONS/WITH_MYC/S71F"
grofile = "confout.gro"
trjfile = "traj_fit.xtc"
#Give selections of two groups between which the native contacts will be computed
sel_ligand = "resname LIGANDRESNAME"    #or e.g. resid 301-400
sel_protein = "protein"   #or e.g. "resid 1-300"
#A contact is considered to exist when the distance between two particles is within the cutoff value (in A)
cutoff = 4.5
print("Input parameters are defined")

Input parameters are defined


In [ ]:
"""""""""""""""""""""""""""""""""
 Calculation of native contacts
"""""""""""""""""""""""""""""""""

#Define the Universe - No need to change any of the following lines
u = mda.Universe(f"{path}/{grofile}", f"{path}/{trjfile}") 
ligand = u.select_atoms(sel_ligand)
protein = u.select_atoms(sel_protein)
results = contacts.Contacts(u, selection = (sel_ligand, sel_protein),
                        refgroup = (ligand, protein), radius = cutoff)

#Iterate through trajectory and perform analysis of "native contacts" Q
#You may want to change the step
results.run(step = 10)  
#Print results - you can comment this out
print("Time [ps]  Fraction of contacts\n", results.timeseries)    

#Print number of averave contacts
average_contacts = np.mean(results.timeseries[:, 1])
print(f'The average fraction of native contacts is {average_contacts}')

"""""""""""""""""""""""""""""""""
  Plot native contacts vs time
"""""""""""""""""""""""""""""""""

#Plot time series q(t)

#You can change the dimensions of the figure or leave this blank to use the default ones

fig, ax = plt.subplots(figsize=(6, 4))    
ax.plot(results.timeseries[:, 0], results.timeseries[:, 1], color='darkred', linewidth=1.5)
#Change the legend
leg = ["YourLegend"]                        
xmin, xmax = ax.get_xlim()
ymin, ymax = ax.get_ylim()
xmin, xmax, ymin, ymax = xmin, xmax, ymin, 0.5    #You can change the limits of the  axes
ax.set_xlim = (xmin, xmax)
ax.set_xlabel("Frame", fontsize=15)
ax.set_ylim(ymin, ymax)
ax.set_ylabel("Fraction of native contacts", fontsize=15)
ax.legend(leg,  loc=1, fancybox=True, shadow=True, fontsize=15)
plt.tick_params(axis='both', bottom=True, top=True, right=True, left=True, which='major', labelbottom=True, labeltop=False, direction='in', labelsize=14, labelrotation=0)
plt.tight_layout()

#Save figure in the same path in .png format
plt.savefig(f'{path}/native_contacts.png', bbox_inches='tight', dpi=600)
plt.show()

### Protein-Ligand contacts

This script will go through each pair of atoms from two groups (e.g. protein and ligand), and if they form a contact, it will add the resids of the protein that are involved in the contact in a list.

You need to provide the path of the input files, the .gro and .xtc (or .trr) files, the group selections (e.g. protein and ligand) and the cutoff.

This code is quite slow, so either select a large step for the trajetory iteration or select part of the protein that you know is likely to be involved in the ligand binding.

In [ ]:
"""""""""""""""""""""""""""""
     Distance function
"""""""""""""""""""""""""""""

#Function that calculates the distance between a pair of particles
def distance(i, j):
    d=norm(i.position-j.position)
    return d, i.resid, j.resid         #returns the distance and the resid the atoms belong to.
  
"""""""""""""""""""""""""""""""""""""""
Calculation of protein-ligand contacts
"""""""""""""""""""""""""""""""""""""""

#Define the Universe - No need to change any of the following lines
u = mda.Universe(f"{path}/{grofile}", f"{path}/{trjfile}") 
ligand = u.select_atoms(sel_ligand)
protein = u.select_atoms(sel_protein)

#You can change the slice of the trajectory or the step
#The code is quite slow, so be cautious if you want to use a smaller step
start, stop, step = 0, u.trajectory.n_frames, 20         

contact_list=[]
for ts in u.trajectory[start:stop:step]:                          #Iterate trajectory frames
    for atom1 in protein:                                         #Go through every possible protein-ligand atom pair
        for atom2 in ligand:
            if distance(atom1, atom2)[0] <= cutoff: #Use distance function and keep only the distances within the cutoff
                print("Resid",distance(atom1, atom2)[1], "Distance =",distance(atom1, atom2)[0], "A")
                contact_list.append((distance(atom1, atom2)[1]))
    print("Time", ts.time/1000, "ns")  
    
#Count and print how many times a residue formed a contact with the ligand
count=Counter(np.array(contact_list))
for x, y in count.items():
    print("Resid", x, "Occurances", y)

#Sort residues in ascending order
residues=np.sort(list(count.keys()))    

#Save contact residues in text file in same directory
np.savetxt(f"{path}/contacts.txt", residues, fmt='%s', delimiter='   ', newline='\n', header=' List of residues that form a contact with the ligand during the simulation', footer='', comments='#', encoding=None)

### Ligand-Contacts minimum distances vs time

This script reads the list of residues that was generated previously and calculates the minimum distances between the ligand and each of the residues vs time.

The minimum distances per residue are saved in a separate .txt file.

A plot is also generated and saved in .png format of all the minimum distances vs time.

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""
Calculate the minimum distances per residue vs time
"""""""""""""""""""""""""""""""""""""""""""""""""""

u = mda.Universe(f"{path}/{grofile}", f"{path}/{trjfile}") 
#You can change the slice of the trajectory or the step
start, stop, step = 0, u.trajectory.n_frames, 20
ligand = u.select_atoms(sel_ligand)
protein = u.select_atoms(sel_protein)
residues=np.loadtxt(f"{path}/contacts.txt", dtype='int', comments=("#", "@"))

for residue in residues:
    mindist_residue=[]
    res=u.select_atoms(f"resid {residue}")
    print("\nResid", str(res.residues.resids),"\n")
    for ts in u.trajectory[start:stop:step]:
        distances=[]
        for atom1 in res:
            for atom2 in ligand:
                distances.append((norm(atom1.position-atom2.position)))
        dist_array=np.array(distances)
        mindist_residue.append((ts.time, np.sort(dist_array)[0]))
        print("Time [ns]  Minimum Distance [A]")
        print(ts.time/1000, np.sort(dist_array)[0])
    np.savetxt(f"{path}/{residue}.txt", mindist_residue, delimiter='   ', newline='\n', header=f'Residue {residue} - Ligand Minimum distance\n Time [ns]        Minimum distance [A]', footer='', comments='@', encoding=None)

In [ ]:
"""""""""""""""""""""
   Read data files
"""""""""""""""""""""

residues=np.loadtxt(f"{path}/contacts.txt", dtype='int', comments=("#", "@"))
#Read minimum distances per residue and vertically stack data in array called 'allmindist'
ylabels=[]
allmindist=np.empty([1, 2])
for residue in residues:
    res=u.select_atoms(f"resid {residue}")
    ylabels.append((str(res.residues.resnames[0])+str(residue)))
    mindist=np.loadtxt(f"{path}/{residue}.txt", comments=("#", "@"))                     
    allmindist=np.vstack((allmindist, mindist))
allmindist=np.delete(allmindist, (0), axis=0)

"""""""""""""""""""""""""""""""""""""""""""""
  Plot minimum distance per residue vs time
"""""""""""""""""""""""""""""""""""""""""""""

#Plot data in barcode subplot
fig, ax=plt.subplots(figsize=(8, len(residues)/5))
nbins=6
n_segs=len(residues)
vmin=1.0
vmax=7.0
pylab.title("Minimum Distances", fontsize=15)
n_steps=allmindist.shape[0]/len(residues)
pylab.pcolormesh(np.array_split(allmindist[:,1], n_segs), cmap='bwr_r', vmin=vmin, vmax=vmax)
cbar = pylab.colorbar()
tick_locator=ticker.MaxNLocator(nbins=nbins)
cbar.locator=tick_locator
cbar.update_ticks()
cbar.set_label("Minimum Distance [$\AA$]", fontsize=15)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.tick_params(labelsize=15) 
pylab.xticks(np.arange(0,n_steps+1, n_steps/10), np.arange(0,101,10), fontsize=13)
pylab.yticks(np.arange(n_segs)+0.5, (ylabels), fontsize=13)
pylab.xlabel("Time [ns]", fontsize=15)
plt.tight_layout()
pylab.savefig(f"{path}/AllMinimumDistancesNew.png", bbox_inches='tight', dpi=600)
pylab.show()

### Contact Lifetime

This script computes the percentage of contact lifetime per residue and saves in a .txt file. 
I also generates and saves in .png format a barcode plot.

In [ ]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""
   Calculate the % of contact lifetime per residue
"""""""""""""""""""""""""""""""""""""""""""""""""""""

#Read list of contact residues that was generated previously
residues=np.loadtxt(f"{path}/contacts.txt", dtype='int', comments=("#", "@"))
print("List of resids that form a contact with the ligand:\n", residues.T)
lifetime_list=[]
print("\nResid", "Lifetime [%]")
for residue in residues:
    count=0
    mindist=np.loadtxt(f"{path}/{residue}.txt", comments=("#", "@"), usecols=(1))
    total=mindist.shape[0]
    for item in mindist:
        if item<=cutoff:
            count+=1
    lifetime=round((count/total)*100, 2)
    print(residue, lifetime)
    lifetime_list.append((residue, lifetime))
np.savetxt(f"{path}/contact_lifetime.txt", lifetime_list, fmt=('%i','%.2f'), delimiter='   ', newline='\n', header='Residue           Contact Lifetime', footer='', comments='@', encoding=None)
    

In [ ]:
"""""""""""""""""""""""""""
   Plot contact lifetime
"""""""""""""""""""""""""""

# Read data from files
residues=np.loadtxt(f"{path}/contact_lifetime.txt", comments=("#", "@"), usecols=(0))
lifetime_list=np.loadtxt(f"{path}/contact_lifetime.txt", comments=("#", "@"), usecols=(1))

fig, ax1 = plt.subplots(figsize=(12,3))

#Figure parameters
bar_width = 0.5
opacity = 1.0
n_groups = residues.shape[0]
index = np.arange(n_groups)
leg = ["YourLegend"]                        #Change the legend
xticklabels=[]
for residue in residues:
    res=u.select_atoms(f"resid {int(residue)}")
    xticklabels.append((str(res.residues.resnames[0])+str(int(residue))))
    
ax1.bar(index, lifetime_list, width=bar_width, alpha=opacity, color='cornflowerblue', edgecolor='mediumblue', linewidth=2.0, align='center', capsize=4.0, fill=True)
ax1.set_xlim([-0.5, n_groups-0.5])
ax1.set_ylim([0.0, 100.0])
ax1.set_xlabel('Residue', fontsize=15)
ax1.set_ylabel('Contact Lifetime [%]', fontsize=15)
ax1.set_xticks(index)
ax1.set_yticks([0.0, 25.0, 50.0, 75.0, 100.0])
ax1.set_xticklabels(xticklabels)
ax1.legend(leg, loc=1, fancybox=True, shadow=True)
plt.tick_params(axis='x', bottom=True, top=True, which='major', labelbottom=True, labeltop=False, direction='in', labelsize=12, labelrotation=35)
plt.tick_params(axis='y', right=True, left=True, which='major', labelleft=True, labelright=False, direction='in', labelsize=14)
fig.tight_layout()

plt.savefig(f"{path}/contact_lifetime.png", bbox_inches='tight', dpi=600)
plt.show()